# Image Classification using Deep Neural Network on Fashion MNIST Dataset

In [ ]:
from keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
train = pd.read_csv("fashion-mnist_train.csv")
test = pd.read_csv("fashion-mnist_test.csv")

In [ ]:
train

In [ ]:
output_mapping = { 0: "T-shirt/Top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 5: "Sandal", 6: "Shirt", 7: "Sneaker",
                 8: "Bag", 9: "Ankle Boot"}

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train.iloc[i, 1:].values.reshape(28,28))
    plt.xlabel(output_mapping[train.iloc[i, 0]])
plt.show()

In [ ]:
train_image = np.array(train.iloc[:,1:])
train_image = train_image/255
train_label = np.array(train.iloc[:, 0])
test_image = np.array(test.iloc[:,1:])
test_image = test_image/255
test_label = np.array(test.iloc[:, 0])

## Deep Neural Network

In [ ]:
model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(784,activation='relu'))
model.add(layers.Dense(400,activation='relu'))
model.add(layers.Dense(300,activation='relu'))
model.add(layers.Dense(200,activation='relu'))
model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(10,activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_image, train_label, epochs = 30, batch_size=32, validation_data=(test_image, test_label))

Epoch 1/30
1875/1875 [==============================] - 50s 26ms/step - loss: 0.5064 - accuracy: 0.8177 - val_loss: 0.4118 - val_accuracy: 0.8544
Epoch 2/30
1875/1875 [==============================] - 43s 23ms/step - loss: 0.3869 - accuracy: 0.8597 - val_loss: 0.3844 - val_accuracy: 0.8554
Epoch 3/30
1875/1875 [==============================] - 45s 24ms/step - loss: 0.3486 - accuracy: 0.8730 - val_loss: 0.3532 - val_accuracy: 0.8632
Epoch 4/30
1875/1875 [==============================] - 45s 24ms/step - loss: 0.3213 - accuracy: 0.8826 - val_loss: 0.3519 - val_accuracy: 0.8684
Epoch 5/30
1875/1875 [==============================] - 43s 23ms/step - loss: 0.3055 - accuracy: 0.8882 - val_loss: 0.3340 - val_accuracy: 0.8806
Epoch 6/30
1875/1875 [==============================] - 45s 24ms/step - loss: 0.2909 - accuracy: 0.8927 - val_loss: 0.3186 - val_accuracy: 0.8866
Epoch 7/30
1875/1875 [==============================] - 46s 25ms/step - loss: 0.2764 - accuracy: 0.8988 - val_loss: 0.3719 -

In [ ]:
predictions = model.predict(test_image)

In [ ]:
model.evaluate(test_image, test_label)

In [ ]:
def visualize_pred(image, pred_prob, actual_label):
    fig, (ax1, ax2) = plt.subplots(figsize=(4,8), ncols=2)
    ax1.imshow(image.reshape(28, 28).squeeze())
    ax1.axis('off')
    pred_label = np.argmax(pred_prob)
    ax1.set_title([output_mapping[actual_label], output_mapping[pred_label]])
    ax2.barh(np.arange(10), pred_prob)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Prediction Probability')
    ax2.set_xlim(0, 1.1)
    plt.tight_layout()

In [ ]:
for i in range(10):
    visualize_pred(test_image[i],predictions[i],test_label[i])

## Deep Neural Network with Dropout

In [ ]:
modelwithdropout = models.Sequential()
modelwithdropout.add(layers.Flatten())
modelwithdropout.add(layers.Dropout(0.25))
modelwithdropout.add(layers.Dense(784,activation='relu'))
modelwithdropout.add(layers.Dropout(0.25))
modelwithdropout.add(layers.Dense(400,activation='relu'))
modelwithdropout.add(layers.Dropout(0.25))
modelwithdropout.add(layers.Dense(300,activation='relu'))
modelwithdropout.add(layers.Dropout(0.25))
modelwithdropout.add(layers.Dense(200,activation='relu'))
modelwithdropout.add(layers.Dropout(0.25))
modelwithdropout.add(layers.Dense(100,activation="relu"))
modelwithdropout.add(layers.Dense(10,activation='softmax'))

In [ ]:
modelwithdropout.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelwithdropout.fit(train_image, train_label, epochs = 30, batch_size=32, validation_data=(test_image, test_label))

In [ ]:
predictionswithdropout = modelwithdropout.predict(test_image)

In [ ]:
modelwithdropout.evaluate(test_image, test_label)

In [ ]:
for i in range(10):
    visualize_pred(test_image[i],predictionswithdropout[i],test_label[i])

## Does Deep Neural Network use Visual Information? What happens if we permute the pixel randomly for all images?

In [ ]:
permutation = torch.randperm(784)

In [ ]:
def blurr_images(arr):
    new_arr = []
    for i in range(len(arr)):
        newimage = arr[i].flatten()
        newimage = newimage[permutation]
        newimage = newimage.reshape(28,28)
        new_arr.append(newimage)
    return np.asarray(new_arr)

In [ ]:
blurr_train_image = blurr_images(train_image)
blurr_test_image = blurr_images(test_image)

In [ ]:
for i in range(4):
    plt.subplot(2,4 , 2*i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_image[i].reshape(28,28))
    plt.xlabel(output_mapping[train_label[i]])
    plt.subplot(2,4 , 2*i + 2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(blurr_train_image[i].reshape(28,28))
    plt.xlabel(output_mapping[train_label[i]])
plt.show()

In [ ]:
blurrmodel = models.Sequential()
blurrmodel.add(layers.Flatten())
blurrmodel.add(layers.Dense(784,activation='relu'))
blurrmodel.add(layers.Dense(400,activation='relu'))
blurrmodel.add(layers.Dense(300,activation='relu'))
blurrmodel.add(layers.Dense(200,activation='relu'))
blurrmodel.add(layers.Dense(100,activation="relu"))
blurrmodel.add(layers.Dense(10,activation='softmax'))

In [ ]:
blurrmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
blurrmodel.fit(blurr_train_image, train_label, epochs = 30, batch_size=32, validation_data=(blurr_test_image, test_label))

In [ ]:
blurrpredictions = blurrmodel.predict(blurr_test_image)

In [ ]:
blurrmodel.evaluate(blurr_test_image, test_label)

In [ ]:
for i in range(5):
    visualize_pred(blurr_test_image[i], blurrpredictions[i],test_label[i])